In [17]:
AGENT_NAME = 'Models/ATLA/PPO agent 100 alts over 0+500 1-17-21.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE = 'PPO agent 100 alts over 0+500 1-17-21 results/surrogates/imitator.pth'
SAVE_DIR = 'PPO agent 100 alts over 0+500 1-17-21 results' + '/'
TEST_NAME = 'FGM_5%'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier

from art.attacks.evasion import FastGradientMethod as FGM

import logging
import pandas as pd
import numpy as np
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [7]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42,
                        )
cols =  env.observation_names

In [9]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [19]:
kwargs = dict(norm=np.inf,
        eps=0.05,
        #num_random_init=0,
        #targeted=False,
        #minimal=False
        )
attack = FGM(estimator=classifier(
                        model=torch.load(SURROGATE),
                        loss=CrossEntropyLoss(), 
                        nb_classes=agent.action_space[0].n, 
                        input_shape=agent.observation_space.shape,
                        device_type='gpu',
                        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                        ),
            **kwargs
             )

In [20]:
logging.basicConfig(level=logging.WARN)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,
                                                      mask=observation_masks,
                                                      time_steps=None)

100%|██████████| 8759/8759 [01:23<00:00, 105.45it/s, ASR=0.528]


In [13]:
display(kpi)

cost_function
annual_peak_average                      1.027318
carbon_emissions_total                   0.915260
cost_total                               0.822845
daily_one_minus_load_factor_average      1.216850
daily_peak_average                       0.964038
electricity_consumption_total            0.927147
monthly_one_minus_load_factor_average    0.993826
ramping_average                          1.234974
zero_net_energy                          1.104975
Name: District, dtype: float64

In [18]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, 
                          index_col=0)
    df_kpis[TEST_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = TEST_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

PPO agent 100 alts over 0+500 1-17-21 results/KPIs.csv updated
